# 3. Blog App
> 기본 기능
- 글 등록
- 열람
- 태그 달기
- 댓글 및 검색 기능
- 콘텐츠 생성 및 편집 기능

## 3.2. 개발 코딩하기
### 3.2.1. 뼈대만들기
- python manage.py startapp blog

In [ ]:
# apps.py : 자동으로 생김
class BlogConfig(AppConfig):
    name = 'blog'

# settings.py
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'bookmark.apps.BookmarkConfig',,
    'blog.apps.Blogconfig'
]

### 3.2.2. 모델 코딩하기

In [ ]:
# models.py
class Post(models.Model):
    title = models.CharField(verbose_name = 'TITLE', max_length = 50)
    # 슬러그: 포스트를 설명하는 핵심 단어 집합, pk로 주로 사용
    slug = models.SlugField('SLUG', unique = True, allow_unicode = True, help_text = 'one word for title alieas')
    description = models.CharField('DESCRIPTION', max_length = 100, blank = True, help_text = 'simple descrition text')
    content = models.TextField('CONTENT')
    create_dt = models.DateTimeField('CREATE DATE', auto_now_add=True)
    modify_dt = models.DateTimeField('MODIFY DATE', auto_now = True)

    class Meta:
        verbose_name = 'post'
        verbose_name_plural = 'posts'
        db_table = 'blog_posts'
        ordering = ('-modify_dt',)

    def __str__(self):
        '''
        객체의 문자열을 객체.title 속성으로 표시
        '''
        return self.title

    def get_absolute_url(self):
        '''
        메소드가 정의된 객체를 지칭하는 url 반환
        '''
        # reverse: url 패턴을 만들어주는 함수
        return reverse('blog:post_detail', args=(self.slug,))

    def get_previous(self):
        '''
        modify_dt컬럼을 기준으로 최신 포스트를 반환하는 함수
        '''
        return self.get_previous_by_modify_dt()

    def get_next(self):
        '''
        modyfy_dt 컬럼을 기준으로 다음 ㅍ포스트를 반환
        '''
        return self.get_next_by_modify_dt()

In [ ]:
# admin.py
from django.contrib import admin
from blog.models import Post
# Register your models here.

@admin.register(Post)
class Postadmin(admin.ModelAdmin):
    '''
    Post 클래스가 admin사이트에서 어떻게 보여줄지 정의
    '''
    # 화면에 출력할 내용
    list_display = ('id', 'title', 'modify_dt')
    # 필더 사이트바
    list_fliter = ('modify_dt')
    # 검색박스 : 어떤 컬럼에서 검색할지 표시    
    search_filed = ('title', 'content')
    #  titie 필드를 사용해러 slug필드가 미리 채워지도록 하기
    prepopulated_fields = {'slug': ('title',)}



#### db에 적용
- python manage.py makemigrations 
- python manage.py migrate

![3-1](images/3-1.png)